# How-to: Compute range-defined quasi-vertical profiles

Daniel Sanchez-Rivas<sup>1</sup> and Miguel A. Rico-Ramirez<sup>1</sup>

<sup>1</sup>Department of Civil Engineering, University of Bristol, Bristol, BS8 1TR, United Kingdom

**This notebook describes the process for generating RD-QVPs.**

### Import relevant packages

In [23]:
import datetime as dt
import os
import numpy as np
import towerpy as tp
import cartopy.crs as ccrs
from tqdm import tqdm


You are using the Towerpy framework, an open source library for
working with polarimetric weather radar data.

If you find our work useful for your research, please consider citing our
following publication:

https://doi.org/10.1016/j.envsoft.105746.2023



### Define working directory, file name and other parameters

In [25]:
RSITE = 'dudwick'
WDIR = f'../datasets/'

SCANS_ELEVS = ['spel1', 'spel2', 'spel3', 'spel4']

START_TIME = dt.datetime(2020, 10, 3, 18, 10)
STOP_TIME = START_TIME+dt.timedelta(minutes=5)

plot_methods = False

We'll need a list containing the path and names of the files to be used. 
Note that four elevation scans will be used to 